In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import seaborn as sns
sns.set()
import pandas as pd
import gc
gc.collect()
%matplotlib inline
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('retina')
from tqdm import tqdm

from pyhive import presto
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
requests.packages.urllib3.disable_warnings()
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import gc
gc.collect()
gc.collect()
import gzip
tqdm.pandas()

/home/ronak/anaconda3/envs/customer_v1/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [4]:
rider_busy_raw_file='1582100338-1582100338-CaptainOrderStatus-riderBusy-20200218-0-0.json.gz'

In [6]:
def parse_time(x):
    return  datetime.strptime(str(x), '%Y%m%d%H%M%S')

with gzip.open(rider_busy_raw_file) as f:
    rider_busy_raw=pd.read_json(rider_busy_raw_file,lines=True)


In [10]:
rider_busy_raw.head()
rider_busy_raw.eventProps.iloc[2]
rider_busy_raw.profile.iloc[2]
rider_busy_raw.deviceInfo.iloc[2]

,eventName,ts,eventProps,profile,deviceInfo
0,CaptainOrderStatus-riderBusy,20200218000032,"{'city': 'Hyderabad', 'pickupClusters': 'Nizam...","{'all_identities': ['imranbaig89@gmail.com', '...","{'osVersion': '9', 'sdkVersion': '30601', 'mak..."
1,CaptainOrderStatus-riderBusy,20200218000032,"{'city': 'Hyderabad', 'pickupClusters': 'ECIL'...",{'all_identities': ['5e4588ab2f2eb56621324042'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak..."
2,CaptainOrderStatus-riderBusy,20200218000035,"{'city': 'Bangalore', 'pickupClusters': 'Sampa...",{'all_identities': ['5c35185f4a267149c769e3e7'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak..."
3,CaptainOrderStatus-riderBusy,20200218000039,"{'city': 'Bangalore', 'pickupClusters': 'BTM',...",{'all_identities': ['5c90fad15e042733c9ced9c4'...,"{'osVersion': '7.0', 'sdkVersion': '30303', 'm..."
4,CaptainOrderStatus-riderBusy,20200218000101,"{'city': 'Hyderabad', 'pickupClusters': 'ECIL'...",{'all_identities': ['5e4588ab2f2eb56621324042'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak..."


{'city': 'Bangalore',
 'pickupClusters': 'Sampagni Rama Nagar',
 'dropClusters': 'Adugodi',
 'paymentType': 'cash',
 'orderType': 'app',
 'clientId': 'self',
 'showOtp': 'true',
 'CT Source': 'API',
 'lat': 0,
 'lng': 0,
 'identity': '5c85cd218c352421eae399a1',
 'serviceType': '57370b61a6855d70057417d1',
 'orderId': '5e4adb0a9a773437e4b7a85e',
 'timeBucket': '00:00-00:30',
 'orderDate': '2020-02-17',
 'otp': '6449'}

{'all_identities': ['5c35185f4a267149c769e3e7',
  'imtiazmncyber@gmail.com',
  'ayeshaparveen9876@gmail.com',
  '5d3d39f09996733edb0519f6'],
 'identity': '5c35185f4a267149c769e3e7,5d3d39f09996733edb0519f6',
 'platform': 'Android',
 'phone': 919916669166,
 'name': 'Mohammed Imtiyaz',
 'email': 'imtiazmncyber@gmail.com',
 'push_token': 'fcm:e_omTxvGns4:APA91bEkEnLwIr2NYRjTRKD6Va9mi0JMDDm1O2i3KJ_8fNCLcP3Cau8Ff4OKJqtd54sfL_LjvYvAnKj2qdvC1YcvymLhc-jQp4cai3S6eGGcI2gZ4XeMsRALVxzKLISXKd5asKgm6eOa'}

{'osVersion': '9',
 'sdkVersion': '30601',
 'make': 'OPPO',
 'model': 'Others',
 'appVersion': '3.10.5',
 'browser': 'MobileApp',
 'dpi': 320,
 'dimensions': {'width': 65, 'height': 126, 'unit': 'mm'}}

In [12]:
from datetime import datetime
def parse_rider_busy_rendered(file_name):
    def get_property_value_for_col(df,source_column,target_column,property_key):
        def get_property(payload,property_key):
            try:
                identity_index=[i for i, s in enumerate(payload) if property_key in s][0]
            except:
                return None
            value=list(payload.values())[identity_index]
            return value
        df.loc[:,target_column]=df[source_column].progress_apply(lambda x: get_property(x,property_key))
        return df
    def parse_time(x):
        return  datetime.strptime(str(x), '%Y%m%d%H%M%S')

    with gzip.open(rider_busy_raw_file) as f:
        rider_busy_raw=pd.read_json(rider_busy_raw_file,lines=True)

    rider_busy_raw['time']=rider_busy_raw['ts'].progress_apply(parse_time)
    source_column='eventProps'
    target_column='order_id'
    property_key='orderId'

    rider_busy_raw=get_property_value_for_col(rider_busy_raw,source_column,target_column,property_key)
    source_column='profile'
    target_column='captain_id'
    property_key='identity'
    rider_busy_raw=get_property_value_for_col(rider_busy_raw,source_column,target_column,property_key)
    
    source_column='deviceInfo'
    target_column='app_version'
    property_key='appVersion'
    rider_busy_raw=get_property_value_for_col(rider_busy_raw,source_column,target_column,property_key)

    return rider_busy_raw

In [15]:
rider_busy=parse_rider_rejected_rendered(rider_busy_raw_file)


100%|██████████| 228665/228665 [00:00<00:00, 319332.38it/s]


In [16]:
rider_busy.head(2)

,eventName,ts,eventProps,profile,deviceInfo,time,order_id,captain_id,app_version
0,CaptainOrderStatus-riderBusy,20200218000032,"{'city': 'Hyderabad', 'pickupClusters': 'Nizam...","{'all_identities': ['imranbaig89@gmail.com', '...","{'osVersion': '9', 'sdkVersion': '30601', 'mak...",2020-02-18 00:00:32,5e4adb912806995062b9f6e6,5bb47306b7078153f3add9e9,3.10.5
1,CaptainOrderStatus-riderBusy,20200218000032,"{'city': 'Hyderabad', 'pickupClusters': 'ECIL'...",{'all_identities': ['5e4588ab2f2eb56621324042'...,"{'osVersion': '9', 'sdkVersion': '30601', 'mak...",2020-02-18 00:00:32,5e4ad8f49a773437e4b7a642,5e4588ab2f2eb56621324042,3.10.5


In [17]:
rider_busy[['eventName','time','order_id','captain_id','app_version']].to_csv('rider_busy_20200218.csv',index=False)